## Solving bidomain equations on rectangular domain with modified FitzHugh-Nagumo model

### Importing required libraries

In [1]:
%cd ..
from src.dynamics_models import *
from src.cell_models import *
from src.utils import *

/home/drvinko/Desktop/Modeliranje-biofizike-srcanog-misica


### Defining a domain, cell model and dynamics model and visualizing initial data

In [2]:
domain = rectangle(2, 2, 20, 20)

In [3]:
class Model(MonodomainModel):
    def initial_V_m(self):
        def value(x):
            mu, alpha = 0.2, 0.03
            return -85 + 35 / (1 + np.exp((x[0] - mu) / alpha))

        self.V_m_n.interpolate(value)
        return self.V_m_n

    def conductivity(self):
        # Muscle fibres
        self.fibers = ufl.as_vector([1, 0])

        # Healthy conductivity
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IL - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)

    def ischemia(self):
        def intersect(X: list[np.ndarray]) -> np.ndarray:
            arr = X[0]
            for x in X:
                arr = np.intersect1d(arr, x)
            return arr

        reduce_factor = 0.1
        locator1 = lambda x: x[0] < 1.5
        locator2 = lambda x: x[0] > 0.5
        locator3 = lambda x: x[1] < 1.5
        locator4 = lambda x: x[1] > 0.5
        cells1 = fem.locate_dofs_geometrical(self.V1, locator1)
        cells2 = fem.locate_dofs_geometrical(self.V1, locator2)
        cells3 = fem.locate_dofs_geometrical(self.V1, locator3)
        cells4 = fem.locate_dofs_geometrical(self.V1, locator4)
        cells = intersect([cells1, cells2, cells3, cells4])
        fun = fem.Function(self.V1)
        fun.x.array[:] = 1
        fun.x.array[cells] = np.full_like(cells, reduce_factor)
        self.M_i = self.M_i * fun
        # self.M_i.x.array[cells] = np.full_like(cells, self.M_i.x.array[cells]/10)
        return fun

In [4]:
cell_model = ReparametrizedFitzHughNagumo(domain)
model = Model(domain, cell_model)

In [5]:
plot_function(
    model.initial_V_m(),
    camera_direction="xy",
    zoom=0.9,
    cmap='PiYG',
    save_to="rectangle_initial_V_m.pdf",
)

Widget(value='<iframe src="http://localhost:45063/index.html?ui=P_0x7f380442cb50_0&reconnect=auto" class="pyvi…

In [6]:
plot_function(
    model.ischemia(),
    camera_direction="xy",
    cmap="RdYlGn",
    save_to="ischemia_ischemia_location.pdf",
)

Widget(value='<iframe src="http://localhost:45063/index.html?ui=P_0x7f37acb03b10_1&reconnect=auto" class="pyvi…

### Solving equations with given parameters

In [7]:
model.solve(
    T=500,
    steps=500,
    save_to="rectangle_MFN.mp4",
    camera_direction="xy",
    zoom=0.9,
    checkpoints=[50, 100, 150, 200, 250, 300, 350, 400, 450, 500],
    checkpoint_file="rectangle_MFN",
)

Solving problem:  60%|█████▉    | 299/500 [00:50<01:00,  3.32it/s]

### Plotting fiber orientations

In [ ]:
plot_vector_field(
    domain,
    lambda x: [
        5 * x[1] ** 2 / np.sqrt((25 * x[1] ** 4 + 1)),
        1 / np.sqrt((25 * x[1] ** 4 + 1)),
        0,
    ],
    0.03,
    0.1,
    camera_direction="xy",
    save_to="rectangle_fibers.pdf",
)

Widget(value='<iframe src="http://localhost:39503/index.html?ui=P_0x7fca82cd3550_1&reconnect=auto" class="pyvi…

ERROR:wslink.protocol:Exception raised
ERROR:wslink.protocol:KeyError('57eb16c3f389c2f659501f32a88d79b1_10201d')
ERROR:wslink.protocol:Traceback (most recent call last):
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/wslink/protocol.py", line 332, in onMessage
    results = func(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/protocols/local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/serializers/synchronization_context.py", line 30, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
                ~~~~~~~~~~~~~~~~~~~~~^^^^^^^
KeyError: '57eb16c3f389c2f659501f32a88d79b1_10201d'

